Playing games with Spuds

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function cost_imitative(a::Spud)
    h_inc = [4, 4, 3, 2, 2, 1, 1, 1, 0, 0] # how much H effectively increases given base H
    c =  6 + h_inc[a.h] * (a.f + a.l) + 2 * ifelse(a.l >= 8, 1, 0)
    if a.f >= 9 && a.l >= 4
        c += 3
    end
    if a.l == 10 && a.p == 10
        c += 8
    end
    return c
end

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function cost_reciprocating(a::Spud)
    # how much F effectively increases given base F (times 2)
    f_inc = [8, 6, 4, 2, 1, 1, 1, 1, 1, 1]
    c = 3 + f_inc[a.f] * div(a.h + a.p + a.r + a.s, 2)
    c = c + ifelse(a.h >= 9, 1, 0) + ifelse(a.p + a.r + a.s >= 16 && a.l >= 10, 4, 0)
    return c
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function cost_romantic(a::Spud)
    # how much L effectively increases given base L
    l_inc = [3, 2, 2, 1, 1, 1, 1, 0, 0, 0]
    c = 5 + l_inc[a.l] * a.h * ifelse(a.h >= 8, 2, 1)
    # nerf Spud("", 9, 1, 4, 10, 10, 10, 999)
    if a.h >= 9 && (a.p + a.r + a.s) >= 30
        c += 3
    end
    return c
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function cost_bibliophile(a::Spud)
    # ban full p/r/s
    if a.p + a.r + a.s >= 30
        return 100
    end
    # how much L effectively increases given base L
    l_inc = [2, 2, 2, 1, 1, 1, 1, 0, 0, 0];
    # for penalizing large H
    # h_inc = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    c = 4 + l_inc[a.l] * a.h
    return c
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function cost_melodious(a::Spud)
    h_inc = [0,0,0,0,0,1,1,2,4,10]
    f_inc = [0,0,0,0,1,1,4,5,9,11]
    c = 5 + f_inc[a.f] + 2 * h_inc[a.h] + ifelse(a.r >= 10, 3, 0)
    # nerf Spud("", 3, 9, 1, 3, 2, 1, 999)
    c += ifelse(a.f >= 9 && (a.p * a.h) >= 9, 3, 0)
    return c
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function cost_iconoclast(a::Spud)
    # ban 10's
    if sum([a.h,a.f,a.l,a.p,a.r,a.s].==10) > 0
        return 100
    end
    l_inc = [0,0,0,0,0,0,0,3,7]
    s_inc = [1,2,3,3,3,3,3,3,3]
    c = 13 + (3 * l_inc[a.l] + 1) * (s_inc[a.s])
    return c
end


function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== 10)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

#df[:, :c2] = df.cost .+ (df.f .+ l_inc[df.l]) .* map(x -> div(x, 5), 4 .+ df.p .+ df.r .+ df.s) .+ 3;
function cost_intoxicating(a::Spud)
    l_inc = [0,0,0,0,0, 0,0,1,2,3]
    c = 3 + (a.f + l_inc[a.l]) * div(4 + a.p + a.r + a.s, 5)
    # nerf Spud("", 1, 4, 10, 2, 10, 3, 999)
    if a.h == 1 && a.f == 4 && a.l == 10 && a.p == 2 && a.r == 10 && a.s == 3
        c += 3
    end
    return c
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function cost_otaku(a::Spud)
    l_inc = [0,0,0,0,0, 0,0,0,0,8]
    c = 0 + l_inc[a.l] * div(7 + a.p + a.r + a.s, 8)
    if maximum([a.p, a.r, a.s]) == 10
        c += a.l
    end
    if c < 6
        c = 6
    end
    return c
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end

function cost_sincere(a::Spud)
    h_inc = [0,0,0,4,5,5,5,5,5,5]
    l_inc = [0,0,1,7,8,9,9,9,9,9]
    s_inc = [0,0,0,0,1,2,2,2,2,2]
    c = (h_inc[a.h] + s_inc[a.s]) * l_inc[a.l];
    if c < 6
        c = 6
    end
    return c
end

function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function cost_cowardly(a::Spud)
    if (a.f < 3) || (a.h > 8)
        return 100
    end
    c = 0
    c2 = cost(a.h + 2, a.f - 2, a.l, a.p, a.r, a.s)
    c0 = cost(a.h, a.f, a.l, a.p, a.r, a.s)
    if c2 > c0
        c = c2 - c0
    end
    f_inc = [0,0,0,0,0,2,2,3,9,10]
    l_inc = [0,0,0,0,0,0,0,0,6,7]
    c += f_inc[a.f] + l_inc[a.l]
    if c < 6
        c = 6
    end
    return c
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_cost = Dict(
    ability_imitative => cost_imitative,
    ability_reciprocating => cost_reciprocating,
    ability_romantic => cost_romantic,
    ability_bibliophile => cost_bibliophile,
    ability_melodious => cost_melodious,
    ability_iconoclast => cost_iconoclast,
    ability_intoxicating => cost_intoxicating,
    ability_otaku => cost_otaku,
    ability_sincere => cost_sincere,
    ability_cowardly => cost_cowardly,
)

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

Dict{Int64, Function} with 10 entries:
  50  => fx_melodious
  20  => fx_reciprocating
  110 => fx_otaku
  10  => fx_imitative
  90  => fx_iconoclast
  30  => fx_romantic
  120 => fx_sincere
  130 => fx_cowardly
  40  => fx_bibliophile
  100 => fx_intoxicating

In [4]:
const MXS = 100
const MXV = 10
const MNV = 1

function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
    #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
    return h*(f+l) + f*(p+r+s)
end

function cost(a::Spud)::Int64
    if a.a1 == ability_none
        return cost(a.h, a.f, a.l, a.p, a.r, a.s)
    else
        return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
    end
end

cost (generic function with 2 methods)

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [119]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, utb)
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    end
end

eval_battle (generic function with 1 method)

In [7]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [8]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [9]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

In [10]:
function upgrade_spud(sp::Spud)::Spud
    h = sp.h
    f = sp.f
    l = sp.l
    p = sp.p
    r = sp.r
    s = sp.s    
    a1 = sp.a1
    check_h = (h == MXV) || (cost(Spud("",h+1,f,l,p,r,s,a1)) > MXS)
    check_f = (f == MXV) || (cost(Spud("",h,f+1,l,p,r,s,a1)) > MXS)
    check_l = (l == MXV) || (cost(Spud("", h,f,l+1,p,r,s,a1)) > MXS)
    check_prs = (p+r+s == 3*MXV) || (cost(Spud("",h,f,l,p+1,r,s,a1)) > MXS)
    while !(check_h && check_f && check_l && check_prs)
        rand_i = rand(1:4)
        if rand_i == 1 && !check_h
            h = h+1
        end
        if rand_i == 2 && !check_f
            f = f+1
        end
        if rand_i == 3 && !check_l
            l = l+1
        end
        if rand_i == 4 && !check_prs
            rand_j = rand([1,1,1,1,1,2,2,2,2,3,3,3])
            if rand_j == 1 && p < MXV
                p += 1
            end
            if rand_j == 2 && r < MXV
                r += 1
            end
            if rand_j == 3 && s < MXV
                s += 1
            end
        end            
        check_h = (h == MXV) || (cost(Spud("",h+1,f,l,p,r,s,a1)) > MXS)
        check_f = (f == MXV) || (cost(Spud("",h,f+1,l,p,r,s,a1)) > MXS)
        check_l = (l == MXV) || (cost(Spud("", h,f,l+1,p,r,s,a1)) > MXS)
        check_prs = (p+r+s == 3*MXV) || (cost(Spud("",h,f,l,p+1,r,s,a1)) > MXS)
    end
    return Spud(sp.name, h, f, l, p, r, s, a1)
end



upgrade_spud (generic function with 1 method)

In [11]:
function sample_library(ss_prob, n_init = 1000000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for a1 in append!([k for k in keys(ability_name)], [ability_none])
        for h in hrange
            if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
                for f in frange
                    if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                        for l in lrange
                            if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                                for p in prange
                                    if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                        for r in rrange
                                            if (cost(h, f, l, p, r, MNV) <= MXS)
                                                for s in srange
                                                    if rand() < ss_prob && (cost(h,f,l,p,r,s) <= MXS)
                                                        ff = Spud("", h,f,l,p,r,s,a1)
                                                        if cost(ff) <= MXS
                                                            ff_i += 1
                                                            ffs[ff_i] = upgrade_spud(ff)
                                                        end
                                                    end
                                                end
                                            end
                                        end                        
                                    end
                                end                        
                            end
                        end
                    end
                end
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

sample_library (generic function with 2 methods)

## Battle code

In [12]:
mutable struct BattleView
    round::Int64
    own::Array{Spud}
    owncount::Array{Int64}
    deployed::Array{Bool}
    obs::Array{Spud}
    obscount::Array{Int64}
end

mutable struct AiPlayer
    genteam::Function
    pick::Function
    create::Function
end

In [194]:
n_copy = 3

function initialize_game(team1::Array{Spud}, team2::Array{Spud})::Array{BattleView}
    teams = [team1, team2]
    bvs = Array{BattleView}(undef, 0)
    for team in teams
        bv = BattleView(0, team, [n_copy for i in 1:length(team)], [false for i in 1:length(team)], [], [])
        append!(bvs, [bv])
    end
    return bvs
end

function observe_fighters(game::Array{BattleView}, choices::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].deployed[choices[i]] = true
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                0
            else
                game[j].obs = append!(game[j].obs, [ff])
                game[j].obscount = append!(game[j].obscount, [game[i].owncount[choices[i]]])
            end
        end
    end
    return game
end

function modify_fighter_counts(game::Array{BattleView}, choices::Array{Int64}, mod::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].owncount[choices[i]] = game[i].owncount[choices[i]] + mod[i]
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                ind = findall(x -> x==ff, game[j].obs)[1]
                game[j].obscount[ind] = game[i].owncount[choices[i]]
            else
                0
            end
        end
    end
    return game
end

function play_combat(game::Array{BattleView}, choices::Array{Int64}, verbose::Bool = false)::Array{BattleView}
    round = game[1].round + 1
    fighters = [game[i].own[choices[i]] for i in 1:2]
    if verbose
        print("Round ")
        print(round)
        print(": #A")
        print(choices[1])
        print(" ")
        print(fighters[1])
        print(" vs #B")
        print(choices[2])
        print(" ")
        println(fighters[2])
    end
    outcome = eval_battle(fighters[1], fighters[2])
    if verbose
        if outcome == 0
            println("Tied!")
        else
            if outcome == 1
                print(fighters[1])
            end
            if outcome == -1
                print(fighters[2])
            end
            println(" wins!")
        end
    end
    game = observe_fighters(game, choices)
    if outcome > -1
        game = modify_fighter_counts(game, choices, [0, -1])
    end
    if outcome < 1
        game = modify_fighter_counts(game, choices, [-1, 0])
    end
    for i in 1:2
        game[i].round = round
    end
    return game
end

function play_add_spud(game::Array{BattleView}, player::Int64, ff::Spud, verbose::Bool = false)::Array{BattleView}
    game[player].own = append!(game[player].own, [ff])
    game[player].owncount = append!(game[player].owncount, [n_copy])
    game[player].deployed = append!(game[player].deployed, [false])
    if verbose
        print("Player ")
        print(player)
        print(" adds #")
        print(["A", "B"][player])
        print(length(game[player].own))
        print(" ")
        println(ff)
    end
    return game
end

function play_is_game_over(game::Array{BattleView})::Bool
    for bv in game
        if sum(bv.owncount) == 0
            return true
        end
    end
    return false
end

play_is_game_over (generic function with 1 method)

## Load precomputed nash eq distribution

In [51]:
nash_env_df = DataFrame(CSV.File("spudsD_mxv10_nash.csv"))
nash_env = df_to_spuds0(nash_env_df)
counts = [parse(Int, s.name[2:end]) for s in nash_env]
nash_env = [rand_rename(ff) for ff in nash_env]
w = counts./sum(counts);

In [204]:
function generate_rand_team(env, counts, nteam)
    cc = cumsum(counts)
    team = Array{Spud}(undef, nteam)
    for i in 1:nteam
        tmp = rand(1:sum(counts))
        ind = sum(cc .< tmp)+1
        team[i] = env[ind]
    end
    return team
end

function ai0_pick(a::BattleView)::Int64
    inds = findall(a.owncount .> 0)
    return rand(inds)
end

function ai0_create(a::BattleView)::Spud
    return generate_rand_team(nash_env, counts, 1)[1]
end

function ai1_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    cands = nash_env[ebs .== maximum(ebs)]
    return rand(cands)
end

lib = sample_library(0.2)

function ai2_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    if maximum(ebs) == length(ops)
        cands = nash_env[ebs .== maximum(ebs)]
        return rand(cands)
    else
        ebs2 = [eval_battle_list(ff, ops) for ff in lib]
        if maximum(ebs2) > maximum(ebs)
            cands = lib[ebs2 .== maximum(ebs2)]
            return rand_rename(rand(cands))
        else
            cands = nash_env[ebs .== maximum(ebs)]
            return rand(cands)
        end
    end
end


ai2_create (generic function with 1 method)

In [71]:
creation_rounds = [5, 10, 15]

function play_game(players::Array{AiPlayer}, verbose::Bool = true)::Int64
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    team1 = ai1.genteam()
    team2 = ai2.genteam()
    game = initialize_game(team1, team2)
    while !play_is_game_over(game)
        # play a round
        choice1 = ai1.pick(game[1])
        choice2 = ai2.pick(game[2])
        game = play_combat(game, [choice1, choice2], verbose)
        if verbose
            for i in 1:2
                println(game[i].owncount)
            end
        end
        if game[1].round in creation_rounds
            for i in 1:2
                ff = ais[i].create(game[i])
                game = play_add_spud(game, i, ff, verbose)
            end
        end
    end
    if sum(game[1].owncount) == 0
        if sum(game[2].owncount) == 0
            return 0
        else
            return -1
        end
    end
    return 1
end

play_game (generic function with 2 methods)

In [98]:
ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai0_create)
ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai0_create)
ais = [ai1, ai2]
@time res = [play_game(ais, false) for i in 1:1000]
mean(res .> 0)

  0.349780 seconds (2.64 M allocations: 271.349 MiB, 12.64% gc time, 9.17% compilation time)


0.525

In [99]:
ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai0_create)
ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai1_create)
ais = [ai1, ai2]
@time res = [play_game(ais, false) for i in 1:1000]
mean(res .> 0)

  8.381915 seconds (143.73 M allocations: 10.483 GiB, 13.14% gc time, 0.34% compilation time)


0.307

In [100]:
ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai1_create)
ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai0_create)
ais = [ai1, ai2]
@time res = [play_game(ais, false) for i in 1:1000]
mean(res .> 0)

  7.885962 seconds (142.93 M allocations: 10.425 GiB, 12.75% gc time, 0.34% compilation time)


0.704

In [102]:
ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai1_create)
ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai1_create)
ais = [ai1, ai2]
@time res = [play_game(ais, false) for i in 1:100]
mean(res .> 0)

  1.704538 seconds (29.02 M allocations: 2.108 GiB, 11.89% gc time, 8.22% compilation time)


0.5

In [189]:
ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai2_create)
ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai1_create)
ais = [ai1, ai2]
@time res = [play_game(ais, false) for i in 1:100]
mean(res .> 0)

 36.099709 seconds (573.41 M allocations: 50.595 GiB, 20.96% gc time, 0.09% compilation time)


0.55

In [120]:
ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai2_create)
ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai1_create)
ais = [ai1, ai2]

2-element Vector{AiPlayer}:
 AiPlayer(var"#215#216"(), ai0_pick, ai2_create)
 AiPlayer(var"#217#218"(), ai0_pick, ai1_create)

In [173]:
@time lib = sample_library(0.1);

  3.444940 seconds (27.77 M allocations: 3.025 GiB, 14.19% gc time)


In [215]:
verbose = true
ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai2_create)
ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai0_pick, ai1_create)
ais = [ai1, ai2]
team1 = ai1.genteam()
team2 = ai2.genteam()
game = initialize_game(team1, team2)
while !play_is_game_over(game)
    # play a round
    println(" ")
    choice1 = ai1.pick(game[1])
    choice2 = ai2.pick(game[2])
    game = play_combat(game, [choice1, choice2], verbose)
    if verbose
        for i in 1:2
            println(game[i].owncount)
        end
    end
    if game[1].round in creation_rounds
        for i in 1:2
            if i==1
                ops = game[2].obs[game[2].obscount .> 0]
                ebs = [eval_battle_list(ff, ops) for ff in nash_env]
                if maximum(ebs) == length(ops)
                    cands = nash_env[ebs .== maximum(ebs)]
                    ff = rand(cands)
                else
                    lib = sample_library(0.005);
                    ebs2 = [eval_battle_list(ff, ops) for ff in lib]
                    if maximum(ebs2) > maximum(ebs)
                        cands = lib[ebs2 .== maximum(ebs2)]
                        ff = rand_rename(rand(cands))
                        #println(rand_rename(ff))
                        append!(saved, [ff])
                    else
                        cands = nash_env[ebs .== maximum(ebs)]
                        ff = rand(cands)
                    end
                end
            else
                ff = ais[i].create(game[i])
            end
            game = play_add_spud(game, i, ff, verbose)
        end
    end
end

 
Round 1: #A3 Spud("Exquisite Magnate Pommes sarladaise", 6, 1, 10, 10, 10, 10, 999) vs #B2 Spud("Respectful Bargeman Rappie pie", 5, 9, 2, 2, 2, 1, 999)
Spud("Respectful Bargeman Rappie pie", 5, 9, 2, 2, 2, 1, 999) wins!
[3, 3, 2, 3, 3]
[3, 3, 3, 3, 3]
 
Round 2: #A4 Spud("Apt Student Gamjajeon", 10, 4, 3, 2, 1, 4, 999) vs #B2 Spud("Respectful Bargeman Rappie pie", 5, 9, 2, 2, 2, 1, 999)
Spud("Apt Student Gamjajeon", 10, 4, 3, 2, 1, 4, 999) wins!
[3, 3, 2, 3, 3]
[3, 2, 3, 3, 3]
 
Round 3: #A2 Spud("Judicious Examiner Chips and dip", 7, 5, 1, 1, 5, 5, 999) vs #B1 Spud("Insatiable Manservant Fondant potatoes", 5, 5, 4, 1, 5, 5, 999)
Spud("Insatiable Manservant Fondant potatoes", 5, 5, 4, 1, 5, 5, 999) wins!
[3, 2, 2, 3, 3]
[3, 2, 3, 3, 3]
 
Round 4: #A1 Spud("Unbending Harpist Truffade", 9, 7, 1, 2, 1, 1, 999) vs #B3 Spud("Innovative Marksman Nikujaga", 5, 4, 4, 6, 1, 8, 999)
Spud("Unbending Harpist Truffade", 9, 7, 1, 2, 1, 1, 999) wins!
[3, 2, 2, 3, 3]
[3, 2, 2, 3, 3]
 
Round 5: #A3 

Spud("Witty Copywriter Pommes sarladaise", 9, 2, 7, 4, 1, 4, 999) wins!
[0, 0, 1, 2, 1, 0, 0, 2]
[0, 0, 0, 0, 3, 2, 2, 2]
 
Round 34: #A3 Spud("Exquisite Magnate Pommes sarladaise", 6, 1, 10, 10, 10, 10, 999) vs #B7 Spud("Witty Copywriter Pommes sarladaise", 9, 2, 7, 4, 1, 4, 999)
Spud("Exquisite Magnate Pommes sarladaise", 6, 1, 10, 10, 10, 10, 999) wins!
[0, 0, 1, 2, 1, 0, 0, 2]
[0, 0, 0, 0, 3, 2, 1, 2]
 
Round 35: #A5 Spud("Engaging Gondolier Kouign patatez", 2, 10, 10, 2, 1, 3, 999) vs #B6 Spud("Cool Illusionist Pommes sarladaise", 10, 1, 8, 4, 1, 5, 999)
Spud("Engaging Gondolier Kouign patatez", 2, 10, 10, 2, 1, 3, 999) wins!
[0, 0, 1, 2, 1, 0, 0, 2]
[0, 0, 0, 0, 3, 1, 1, 2]
 
Round 36: #A5 Spud("Engaging Gondolier Kouign patatez", 2, 10, 10, 2, 1, 3, 999) vs #B6 Spud("Cool Illusionist Pommes sarladaise", 10, 1, 8, 4, 1, 5, 999)
Spud("Engaging Gondolier Kouign patatez", 2, 10, 10, 2, 1, 3, 999) wins!
[0, 0, 1, 2, 1, 0, 0, 2]
[0, 0, 0, 0, 3, 0, 1, 2]
 
Round 37: #A8 Spud("Bibliophi

In [216]:
spuds_to_df(append!(game[1].own, game[2].own))

Row,name,h,f,l,p,r,s,a1
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,Unbending Harpist Truffade,9,7,1,2,1,1,999
2,Judicious Examiner Chips and dip,7,5,1,1,5,5,999
3,Exquisite Magnate Pommes sarladaise,6,1,10,10,10,10,999
4,Apt Student Gamjajeon,10,4,3,2,1,4,999
5,Engaging Gondolier Kouign patatez,2,10,10,2,1,3,999
6,Romantic* Company secretary Batata vada,7,2,4,3,10,10,30
7,Vibrant Laundress (also Lavendar) Tashmijab,10,2,6,1,4,5,999
8,Bibliophile* Paleontologist Ajiaco,7,2,4,10,10,3,40
9,Insatiable Manservant Fondant potatoes,5,5,4,1,5,5,999


In [178]:
# custom code to check for special spuds
verbose = false
saved = Array{Spud}(undef, 0)

for iii in 1:100
    team1 = ai1.genteam()
    team2 = ai2.genteam()
    game = initialize_game(team1, team2)
    while !play_is_game_over(game)
        # play a round
        choice1 = ai1.pick(game[1])
        choice2 = ai2.pick(game[2])
        game = play_combat(game, [choice1, choice2], verbose)
        if verbose
            for i in 1:2
                println(game[i].owncount)
            end
        end
        if game[1].round in creation_rounds
            for i in 1:2
                if i==1
                    ops = game[2].obs[game[2].obscount .> 0]
                    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
                    if maximum(ebs) == length(ops)
                        cands = nash_env[ebs .== maximum(ebs)]
                        ff = rand(cands)
                    else
                        lib = sample_library(0.005);
                        ebs2 = [eval_battle_list(ff, ops) for ff in lib]
                        if maximum(ebs2) > maximum(ebs)
                            cands = lib[ebs2 .== maximum(ebs2)]
                            ff = rand(cands)
                            println(rand_rename(ff))
                            append!(saved, [ff])
                        else
                            cands = nash_env[ebs .== maximum(ebs)]
                            ff = rand(cands)
                        end
                    end
                else
                    ff = ais[i].create(game[i])
                end
                game = play_add_spud(game, i, ff, verbose)
            end
        end
    end
end

Spud("Otaku* Flavorist Brændende kærlighed", 5, 1, 10, 10, 7, 4, 110)
Spud("Sincere* Consultant Cheese fries", 5, 1, 7, 4, 5, 5, 120)
Spud("Impressive Impersonator Fritter roll", 6, 4, 8, 4, 1, 2, 999)
Spud("Romantic* Etymologist German fries", 3, 3, 4, 10, 10, 3, 30)
Spud("Romantic* Etymologist Llapingacho", 3, 3, 5, 10, 10, 2, 30)
Spud("Tactful Porter Brændende kærlighed", 2, 9, 5, 1, 5, 2, 999)
Spud("Imitative* Soil scientist Ragda pattice", 1, 1, 10, 10, 10, 10, 10)
Spud("Romantic* Vice president Suan la tu dou si", 7, 1, 8, 10, 10, 10, 30)
Spud("Charming Lighting technician Tashmijab", 2, 6, 10, 6, 3, 2, 999)
Spud("Romantic* Hydraulic engineer Batata vada", 8, 1, 5, 10, 10, 10, 30)
Spud("Appreciative Dictator Perkedel / begedil", 4, 5, 10, 3, 2, 3, 999)
Spud("Reciprocating* Real estate investor Cheese fries", 3, 4, 9, 2, 3, 6, 20)
Spud("Intoxicating* Stewardess (or Steward) Potato babka", 1, 7, 9, 2, 1, 6, 100)
Spud("Intoxicating* Dentist Truffade", 1, 3, 10, 2, 10, 8, 100)
Spud("

In [179]:
sum(spuds_to_df(saved).a1 .!= 999)

18

In [165]:
#18 out of 100, for (indep) libr frac 0.005
#23 out of 100, for (indep) libr frac 0.01

#35 out of 100, for libr frac 0.1
#[34, 35], 32 out of 100, for libr frac 0.2